In [6]:
import sqlite3
import pandas as pd

conn = sqlite3.connect(r'C:\Users\foxgh\OneDrive\Área de Trabalho\SQL Playground\Script\data-delivery.db')
cursor = conn.cursor()

In [7]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()
print("Tabelas no banco de dados:", tabelas)


Tabelas no banco de dados: [('vendas',), ('delivery',)]


In [ ]:
query = """
    select 
            *
    from delivery
    """

df = pd.read_sql_query(query, conn)

df

,Order_ID,ID,Cliente,Restaurante,Categoria_Pedido,Valor_Pedido,Taxa_Entrega,Horario_Pedido,Metodo_Pagamento,Tempo_Entrega_Min,Status_Pedido,Avaliacao
0,ORD44734,1,Cliente_1,Restaurante_29,Hambúrguer,286.01,2.19,2024-01-01 11:00:00,Dinheiro,49,Em andamento,5
1,ORD65054,2,Cliente_2,Restaurante_24,Bebidas,161.94,10.37,2024-01-01 11:01:00,Boleto,52,Entregue,1
2,ORD54901,3,Cliente_3,Restaurante_25,Churrasco,161.75,7.83,2024-01-01 11:02:00,PIX,24,Entregue,4
3,ORD53467,4,Cliente_4,Restaurante_44,Hambúrguer,141.42,10.02,2024-01-01 11:03:00,Cartão de Crédito,42,Em andamento,4
4,ORD27664,5,Cliente_5,Restaurante_33,Bebidas,188.26,9.06,2024-01-01 11:04:00,Boleto,11,Cancelado,5
...,...,...,...,...,...,...,...,...,...,...,...,...
395,ORD88664,196,Cliente_196,Restaurante_47,Salada,16.52,2.61,2024-01-01 14:15:00,Boleto,38,Entregue,3
396,ORD25288,197,Cliente_197,Restaurante_18,Bebidas,279.41,4.39,2024-01-01 14:16:00,Boleto,51,Entregue,1
397,ORD94505,198,Cliente_198,Restaurante_36,Pizza,249.21,9.37,2024-01-01 14:17:00,Boleto,37,Em andamento,3
398,ORD80214,199,Cliente_199,Restaurante_21,Marmita,290.65,10.67,2024-01-01 14:18:00,Carteira Digital,55,Entregue,1


In [19]:
query = """
    select 
        categoria_pedido,
        count(distinct order_id) as qtd_pedidos,
        sum(valor_pedido) as total_vendas,
        round(avg(taxa_entrega),2) as media_taxa_entrega
    from delivery
    group by categoria_pedido
    order by qtd_pedidos desc
    
    """

df = pd.read_sql_query(query, conn)

df

,Categoria_Pedido,qtd_pedidos,total_vendas,media_taxa_entrega
0,Marmita,32,9463.48,9.16
1,Vegano,28,7841.80,7.15
2,Sushi,25,7318.38,8.50
3,Salada,25,7154.48,7.99
4,Churrasco,24,8269.74,9.02
5,Pizza,23,7241.22,7.63
6,Bebidas,22,7501.56,9.57
7,Hambúrguer,21,7197.70,9.11


In [44]:
# Média da taxa de entrega por categoria
query = """

with t1 as (
    select 
        categoria_pedido,
        count(distinct order_id) as qtd_pedidos,
        round(avg(taxa_entrega),2) as media_taxa_entrega       
    from delivery
    where status_pedido = 'Entregue'

    group by categoria_pedido
)

select
    categoria_pedido,
    qtd_pedidos,
    media_taxa_entrega,
    case when 
        media_taxa_entrega >= 9 then 'Acima da média'
        when media_taxa_entrega between 7.63 and 8.5  then 'Está na média'
    else 'Abaixo da média' end as comparativo
from t1
group by categoria_pedido
order by media_taxa_entrega desc
    """

df = pd.read_sql_query(query, conn)

df

,categoria_pedido,qtd_pedidos,media_taxa_entrega,comparativo
0,Hambúrguer,9,9.92,Acima da média
1,Marmita,14,9.60,Acima da média
2,Pizza,5,9.33,Acima da média
3,Bebidas,8,9.29,Acima da média
4,Churrasco,6,8.09,Está na média
5,Sushi,7,7.60,Abaixo da média
6,Salada,7,6.05,Abaixo da média
7,Vegano,12,5.83,Abaixo da média


In [ ]:
# Métodos de pagamento mais utilizados

query = """
    select 
        metodo_pagamento,
        count(distinct order_id) as qtd_pedidos,
        count(distinct order_id) * 100.0 / (select count(distinct order_id) from delivery) as perc_pedidos

    from delivery
    where status_pedido = 'Entregue'
    group by metodo_pagamento
    """

df = pd.read_sql_query(query, conn)

df

,Metodo_Pagamento,qtd_pedidos,perc_pedidos
0,Boleto,21,10.5
1,Carteira Digital,13,6.5
2,Cartão de Crédito,10,5.0
3,Dinheiro,14,7.0
4,PIX,10,5.0


In [66]:

query = """

with t1 as (
    select 
        categoria_pedido,
        count(distinct order_id) as qtd_pedidos,
        
        round(avg(tempo_entrega_min),2) as media_tempo_entrega,
        max(tempo_entrega_min) as max_tempo_entrega,
        min(tempo_entrega_min) as min_tempo_entrega

    from delivery
    where status_pedido in ('Entregue', 'Em Andamento')
    group by metodo_pagamento
    )
    
select
    categoria_pedido,
    qtd_pedidos,
    media_tempo_entrega,
    max_tempo_entrega,
    min_tempo_entrega,
    
    case when media_tempo_entrega > 35 then 'Acima da média'
        when media_tempo_entrega between 30 and 35  then 'Está na média'
    else 'Abaixo da média' end as comparativo

from t1
group by categoria_pedido
order by media_tempo_entrega desc
    """

df = pd.read_sql_query(query, conn)

df

,categoria_pedido,qtd_pedidos,media_tempo_entrega,max_tempo_entrega,min_tempo_entrega,comparativo
0,Hambúrguer,13,37.31,59,14,Acima da média
1,Vegano,21,36.90,59,11,Acima da média
2,Marmita,14,34.00,57,16,Está na média
3,Salada,10,27.70,50,10,Abaixo da média


In [96]:

query = """

with t1 as (
    select 
        restaurante,
        count(distinct order_id) as qtd_pedidos_dinheiro,
        sum(valor_pedido) as total_vendas_dinheiro
    from delivery
    where metodo_pagamento = 'Dinheiro'
    group by restaurante
),

t2 as (
    select
        restaurante,
        count(distinct order_id) as qtd_pedidos_cartao,
        sum(valor_pedido) as total_vendas_cartao
    from delivery
    where metodo_pagamento in ('Cartão de Crédito', 'Carteira Digital')
    group by restaurante
),

t3 as (
    select
        restaurante,
        count(distinct order_id) as qtd_pedidos_pix,
        sum(valor_pedido) as total_vendas_pix
    from delivery
    where metodo_pagamento in ('Pix','Boleto')
    group by restaurante
),

t4 as (
    select 
        t1.restaurante,
        coalesce(t1.qtd_pedidos_dinheiro, 0) + 
        coalesce(t2.qtd_pedidos_cartao, 0) + 
        coalesce(t3.qtd_pedidos_pix, 0) as total_pedidos
    from t1 
    left join t2 on t1.restaurante = t2.restaurante
    left join t3 on t1.restaurante = t3.restaurante
)

select * from t4
order by total_pedidos desc

    """

df = pd.read_sql_query(query, conn)

df

,restaurante,total_pedidos
0,Restaurante_41,8
1,Restaurante_17,6
2,Restaurante_29,6
3,Restaurante_47,6
4,Restaurante_1,5
5,Restaurante_13,5
6,Restaurante_18,5
7,Restaurante_21,5
8,Restaurante_22,5
9,Restaurante_27,5
